In [ ]:
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp

In [ ]:
dataDir = os.path.join(os.path.pardir,"data","Scotland")

In [ ]:
dat = pd.read_csv(os.path.join(dataDir,"Scotland_Temperatures.csv"),sep=",",na_values="MISS")
dat.head()

In [ ]:
dat = dat.iloc[np.where(~np.isnan(dat["January_temp"]))[0],:]
temperatures = gl.Db()
names =dat.columns.to_list()
temperatures[names] = np.array(dat[names])
temperatures.setLocator(["Longitude","Latitude"],gl.ELoc.X)
temperatures.setLocator("January_temp",gl.ELoc.Z)
temperatures.setLocator("Elevation",gl.ELoc.F)
temperatures["sel"] = ~np.isnan(temperatures["*temp"])
temperatures.setLocator("sel",gl.ELoc.SEL)
temperatures

In [ ]:
MNT = pd.read_csv(os.path.join(dataDir,"Scotland_Elevations.csv"),sep=",")
MNT.head()

In [ ]:
mnt = gl.Db()
names =MNT.columns.to_list()
mnt[names] = np.array(MNT[names])
mnt.setLocator(["Longitude","Latitude"],gl.ELoc.X)

grid = gl.Db()
#Est-ce qu'on ne pourrait pas migrer toutes les variables (en option) dès l'appel à resetCoveringDb?
grid.resetCoveringDb(mnt,nodes = [81,137])
gl.migrate(mnt,grid,"Elevation",namconv=gl.NamingConvention(""))
gl.migrate(mnt,grid,"inshore",namconv = gl.NamingConvention(""))
grid.setLocator("Elevation",gl.ELoc.F)
grid.setLocator("inshore",gl.ELoc.SEL)
gp.grid(grid,"Elevation")

#Put Nan to 0 for selection (it should become useless in future version)
u = grid["inshore"]
u[np.isnan(grid["inshore"])] = 0.
grid["inshore"] = u

In [ ]:
v = gl.VarioParam()
dir1 = gl.DirParam(2,18,25)
v.addDirs(dir1)
vario = gl.Vario(v,temperatures)
md = gl.Model.createFromDb(temperatures)
md.addDrift(gl.Drift1(md.getContext()))
md.addDrift(gl.DriftF(md.getContext()))
vario.compute(model=md)
ax=gp.vario(vario)


In [ ]:
model = gl.Model.createFromDb(temperatures)
model.setDriftList(md.getDriftList())

structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K,gl.ECov.BESSEL_K]
structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K]

consNug = gl.ConsItem.define(gl.EConsElem.SILL,0, type = gl.EConsType.UPPER,value = 0.1)

cons1P = gl.ConsItem.define(gl.EConsElem.PARAM,1, type = gl.EConsType.EQUAL,value = 1)
cons1Rm = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.LOWER,value = 100)
cons1RM = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.UPPER,value = 350)

cons2P = gl.ConsItem.define(gl.EConsElem.PARAM,2, type = gl.EConsType.EQUAL,value = 2)
cons2Rm = gl.ConsItem.define(gl.EConsElem.RANGE,2, type = gl.EConsType.LOWER,value = 100)
cons2RM = gl.ConsItem.define(gl.EConsElem.RANGE,2, type = gl.EConsType.UPPER,value = 400)

a = gl.Constraints()
a.addItem(consNug)
a.addItem(cons1P)
a.addItem(cons1Rm)
a.addItem(cons1RM)
a.addItem(cons2P)
a.addItem(cons2Rm)
a.addItem(cons2RM)

err = model.fit(vario,structs,constraints=a)
ax = gp.varmod(vario,model,title="Vario of the residuals")

In [ ]:
model

In [ ]:
spde = gl.SPDE(model,grid,temperatures,gl.ESPDECalcMode.KRIGING)

In [ ]:
coeffs = spde.getCoeffs()

In [ ]:
ax = gp.correlation(temperatures, "Elevation", "*temp", asPoint=True)
if len(coeffs)>1:
    plt.plot([0,400], [coeffs[0],coeffs[0]+coeffs[1]*400])

In [ ]:
model

In [ ]:
fileM = os.path.join("/home","ndesassis","gstlearn","gstlearn","doc","data","Scotland","model.ascii")
err = model.dumpToNF(fileM)
fileG = os.path.join("/home","ndesassis","gstlearn","gstlearn","doc","data","Scotland","grid.ascii")
err = grid.dumpToNF(fileG)
fileT = os.path.join("/home","ndesassis","gstlearn","gstlearn","doc","data","Scotland","temperatures.ascii")
err = temperatures.dumpToNF(fileT)
fileV = os.path.join("/home","ndesassis","gstlearn","gstlearn","doc","data","Scotland","vario.ascii")
err = vario.dumpToNF(fileV)
#gl.Model.createFromNF(fileM)
#gl.Db.createFromNF(fileG)
#gl.Db.createFromNF(fileT)
#gl.Vario.createFromNF(fileV)

In [ ]:
spde.compute()
iatt = spde.query(grid)

In [ ]:
model

In [ ]:
err = gl.kriging(temperatures,grid,model,gl.NeighUnique.create(2))

In [ ]:
ax = gp.grid(grid,"spde*")
ax = gp.grid(grid,"*estim")

In [ ]:
ax = gp.correlation(grid,"*estim","spde*",asPoint=True, diagLine=True)
ax = gp.correlation(grid,"*estim","spde*", diagLine=True)